In [1]:
import anndata
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
group_name = "MOp"

In [3]:
region = group_name

In [5]:
tartype = {
    "IT": [
        "PFC",
        "MOp",
        "SSp",
        "ACA",
        "AI",
        "AUDp",
        "RSP",
        "PTLp",
        "VISp",
        "MOB",
        "ENT",
        "HPF",
        "PIR",
        "AMY",
        "STR",
        "PAL",
    ],
    "ET": ["TH", "HY", "SC", "VTA", "P", "MY"],
    "CB": ["CBN", "CBX"],
}
tarmap = {yy: xx for xx in tartype for yy in tartype[xx]}

In [6]:
source_list = [
    "MOp",
    "SSp",
    "ACA",
    "AI",
    "RSP",
    "AUD",
    "PTLp",
    "VIS",
    "ENT",
    "CAp",
    "PIRa",
    "PIRp",
    "MOB",
    "AON",
    "PAL",
    "AMY",
    "THl",
    "THp",
    "HY",
    "MRN",
    "VTA",
    "PAG",
    "IC",
]

In [7]:
adata = anndata.read_h5ad(
    f"/home/jzhou_salk_edu/sky_workdir/230129_targetpair_roc/data/{region}_RS2_geneCH.h5ad"
)
adata

AnnData object with n_obs × n_vars = 2568 × 9906
    obs: 'mCCCFrac', 'mCGFrac', 'mCHFrac', 'FinalmCReads', 'Plate', 'PlateNormCov', 'Sample', 'SubRegion', 'DissectionRegion', 'Target', 'Sex', 'L1', 'L2', 'L3', 'L4', 'L1_annot', 'PassTargetFilter'
    var: 'chrom', 'end', 'start'
    obsm: 'X_pca'

In [8]:
adata.obs["TargetType"] = adata.obs["Target"].map(tarmap)
adata = adata[adata.obs["TargetType"] != "CB"].copy()
adata.obs["TargetType"].astype(str).value_counts()

IT    1409
ET    1159
Name: TargetType, dtype: int64

In [9]:
label = (adata.obs["TargetType"] == "IT").values.astype(int)
result = {}
clf = LogisticRegression()
clf.fit(adata.X, label)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [12]:
for xx in source_list:
    if xx!=region:
        adata = anndata.read_h5ad(f"/home/jzhou_salk_edu/sky_workdir/230129_targetpair_roc/data/{xx}_RS2_geneCH.h5ad")
        adata.obs["TargetType"] = adata.obs["Target"].map(tarmap)
        adata = adata[adata.obs["TargetType"] != "CB"].copy()
        adata.obs["TargetType"].astype(str).value_counts()
        label = (adata.obs["TargetType"] == "IT").values.astype(int)
        pred = clf.predict_proba(adata.X)[:, 1]
        result[xx] = roc_auc_score(label, pred)
        print(xx)

SSp
ACA
AI
RSP
AUD
PTLp
VIS
ENT
CAp
PIRa
PIRp
MOB
AON
PAL
AMY
THl
THp
HY
MRN
VTA
PAG
IC


In [14]:
result = pd.DataFrame({region:result}).T
result.to_hdf(f'/home/jzhou_salk_edu/sky_workdir/230203_ITvsET/result/{region}-crosssource.hdf', key='data')
